# Aligning Data to Darwin Core - Sampling Event with Measurement or Fact using Python
Matt Biddle

November 9, 2020

# General information about this notebook
This notebook was created for the IOOS DMAC Code Sprint Biological Data Session
The data in this notebook were created specifically as an example and meant solely to be
illustrative of the process for aligning data to the biological data standard - Darwin Core.
These data should not be considered actually occurrences of species and any measurements
are also contrived. This notebook is meant to provide a step by step process for taking
original data and aligning it to Darwin Core

This notebook is a python implementation of the R notebook [IOOS_DMAC_DataToDWC_Notebook_event.R](https://github.com/ioos/bio_data_guide/blob/master/Standardizing%20Marine%20Biological%20Data/datasets/example_script_with_fake_data/IOOS_DMAC_DataToDwC_Notebook_event.R)

In [ ]:
import pandas as pd
import pyworms # pip install git+git://github.com/iobis/pyworms.git
import numpy as np
import uuid
import csv

Read in the raw data file.

In [ ]:
url = "http://www.neracoos.org/erddap/tabledap/WBTS_CFIN_2005_2017.csv"
df = pd.read_csv(url, header=[0])

First we need to to decide if we will provide an occurrence only version of the data or
a sampling event with measurement or facts version of the data. Occurrence only is easier
to create. It's only one file to produce. However, several pieces of information will be
left out if we choose that option. If we choose to do sampling event with measurement or
fact we'll be able to capture all of the data in the file creating a lossless version.
Here we decide to use the sampling event option to include as much information as we can.

First let's create the eventID and occurrenceID in the original file so that information
can be reused for all necessary files down the line.

In [ ]:
df['eventID'] = df[['Cruise_Identification_Tag', 'Station_ID']].apply(lambda x: '_'.join(x.astype(str)), axis=1)
df['occurrenceID'] = uuid.uuid4()

# Event file

We will need to create three separate files to comply with the sampling event format.
We'll start with the event file but we only need to include the columns that are relevant
to the event file.

In [ ]:
event = df[['time', 'latitude', 'longitude', 'Station_ID', 'CRUISE_ID', 'NET_DEPTH', 'STATION_DEPTH', 'eventID']].copy()

Next we need to rename any columns of data that match directly to Darwin Core. We know
this based on our crosswalk spreadsheet CrosswalkToDarwinCore.csv

In [ ]:
event['decimalLatitude'] = event['latitude']
event['decimalLongitude'] = event['longitude']
event['minimumDepthInMeters'] = event['NET_DEPTH']
event['maximumDepthInMeters'] = event['NET_DEPTH']

Let's see how it looks:

In [ ]:
event.head()

We will also have to add any missing required fields

In [ ]:
event['basisOfRecord'] = 'HumanObservation'

#This is a guess
event['geodeticDatum'] = 'EPSG:4326 WGS84'

Then we'll remove any columns that we no longer need to clean things up a bit.

In [ ]:
event.drop(
    columns=['latitude', 'longitude', 'Station_ID', 'CRUISE_ID', 'NET_DEPTH'],
    inplace=True)

We have too many repeating rows of information. We can pare this down using eventID which
is a unique identifier for each sampling event in the data- which is six, three transects
per site.

In [ ]:
event.drop_duplicates(
    subset='eventID',
    inplace=True)

event.head(6)

Finally we write out the event file

In [ ]:
event.to_csv(
    'results/WBTS_CFIN_2005_2017_event_frompy.csv',
    header=True,
    index=False,)

# Occurrence file
Next we need to create the occurrence file. We start by examining the structure (columns) of the source data. The goal here is to assess what kind of conversion (if any) will be necessary for Darwin Core alignment. 

In [ ]:
df.head(10)

In this case, the `Calanus_finmarchicus` columns need to be converted into a more suitable format. We need to iterate through the existing data row by row - the goal is to create five new columns: `scientificName`, `lifeStage`, `sex`, `occuranceStatus`, & `individualCount`.

We start by isolating the records that have valid data. We define the columns we want to check against as `target_data_columns`, and then create a new dataframe `calanus_records` by retaining only records where at least one of the columns has a value of NOT `0` AND NOT `NaN`.

We also drop the second row, which contains unit information to avoid confusing the parser. 

In [ ]:
target_data_columns = ['Calanus_finmarchicus_N',
                       'Calanus_finmarchicus_CI',
                       'Calanus_finmarchicus_CII',
                       'Calanus_finmarchicus_CIII',
                       'Calanus_finmarchicus_CIV',
                       'Calanus_finmarchicus_CV',
                       'Calanus_finmarchicus_F',
                       'Calanus_finmarchicus_M']

calanus_records = df.loc[(pd.notna(df[target_data_columns]) & (df[target_data_columns] != 0)).all(1)]

# drop units row from calanus records
calanus_records = calanus_records.iloc[1:, :]

The challenge is that, in its current form, each row actually represents between 0 and 8 discrete occurances. This isn't suitable for Darwin Core, so we need to read each row, and then split its data into new records, each representing an occurance event. This is a little tricky, so we'll create a helper method `enumerate_row` which takes a row (a `pandas.Series` object in practice) and makes the appropriate transformations. 

In [ ]:
def enumerate_row(row, field):
    # expands rows which contain multiple observations into discrete records
    row_data = row[1]
    calanus_count = row_data[field]

    # convert to dict so we can mutate
    enumerated_row = row_data.to_dict()

    split_species = field.rsplit('_', 1)
    scientific_name = split_species[0].replace('_', ' ')
    life_stage = split_species[1]

    # add count of specified species as a new column
    enumerated_row['individualCount'] = calanus_count
    enumerated_row['scientificName'] = scientific_name
    # we're only processing records with occurances
    enumerated_row['occurrenceStatus'] = 'present'

    life_stage = field.rsplit('_', 1)[1]
    enumerated_row['lifeStage'] = life_stage if life_stage != 'F' and life_stage != 'M' else 'adult'

    if life_stage == 'F':
        enumerated_row['sex'] = 'female'
    elif life_stage == 'M':
        enumerated_row['sex'] = 'male'
    else:
        enumerated_row['sex'] = 'NA'

    return enumerated_row

The next step is to loop through the target data. The top-level control variable is the list of the columns we wish to enumerate, so we will look for each target column in each row of the dataset. 

*note*: This operation could easily become costly depending on the number of rows and target columns

In [ ]:
enumerated_rows = []
# loop through target column list, for each, select all records (via loc) where a given column has a value of >0
for field in target_data_columns:

    # returns df with all records where there is an occurance of the given calanus
    current_df = calanus_records.loc[pd.to_numeric(calanus_records[field]) > 0]

    # now enumerate each input row, extracting the values
    for row in current_df.iterrows():

        flipped_row = enumerate_row(row, field)

        # delete other calanus records from flipped row
        for k in target_data_columns:
            flipped_row.pop(k, None)

        enumerated_rows.append(flipped_row)

A little bit of clean up:

In [ ]:
# now convert the list of dicts into a dataframe
output_frame = pd.DataFrame.from_dict(enumerated_rows)

# sort by time, ascending
output_frame.sort_values(by='time', ascending=True, inplace=True)

Now our data should be in a more suitable fromat, so we can proceed. 

We start by creating a new occurrence data frame with the relevant fields.

In [ ]:
occurrence = output_frame[['scientificName', 'eventID', 'occurrenceID', 'individualCount', 'occurrenceStatus', 'lifeStage', 'sex']].copy()

## Taxonomic Name Matching
A requirement for OBIS is that all scientific names match to the World Register of
Marine Species (WoRMS) and a scientificNameID is included. A scientificNameID looks
like this "urn:lsid:marinespecies.org:taxname:275730" with the last digits after
the colon being the WoRMS aphia ID. We'll need to go out to WoRMS to grab this
information.

Create a lookup table of unique scientific names

In [ ]:
lut_worms = pd.DataFrame(
    columns=['scientificName'],
    data=occurrence['scientificName'].unique())

Add the columns that we can grab information from WoRMS including the required scientificNameID.

In [ ]:
headers = ['acceptedname', 'acceptedID', 'scientificNameID', 'kingdom', 'phylum',
           'class', 'order', 'family', 'genus', 'scientificNameAuthorship', 'taxonRank']

for head in headers:
    lut_worms[head] = ''

Taxonomic lookup using the library [pyworms](https://github.com/iobis/pyworms)

In [ ]:
for index, row in lut_worms.iterrows():
    print('Searching for scientific name = %s' % row['scientificName'])
    resp = pyworms.aphiaRecordsByMatchNames(row['scientificName'])[0][0]
    lut_worms.loc[index, 'acceptedname'] = resp['valid_name']
    lut_worms.loc[index, 'acceptedID'] = resp['valid_AphiaID']
    lut_worms.loc[index, 'scientificNameID'] = resp['lsid']
    lut_worms.loc[index, 'kingdom'] = resp['kingdom']
    lut_worms.loc[index, 'phylum'] = resp['phylum']
    lut_worms.loc[index, 'class'] = resp['class']
    lut_worms.loc[index, 'order'] = resp['order']
    lut_worms.loc[index, 'family'] = resp['family']
    lut_worms.loc[index, 'genus'] = resp['genus']
    lut_worms.loc[index, 'scientificNameAuthorship'] = resp['authority']
    lut_worms.loc[index, 'taxonRank'] = resp['rank']

Merge the lookup table of unique scientific names back with the occurrence data.

In [ ]:
occurrence = pd.merge(occurrence, lut_worms, how='left', on='scientificName')

We're going to remove any unnecessary columns to clean up the file

In [ ]:
occurrence.drop(
    columns=['scientific name', 'percent cover'],
    inplace=True)

Quick look at what we have before we write out the file

In [ ]:
occurrence.head()

Write out the file.

In [ ]:
# sort the columns on scientificName
occurrence.sort_values('scientificName', inplace=True)
# reorganize column order to be consistent with R example:
columns = ["scientificName","eventID","occurrenceID","occurrenceStatus","acceptedname","acceptedID",
           "scientificNameID","kingdom","phylum","class","order","family","genus","scientificNameAuthorship",
           "taxonRank"]

occurrence.to_csv(
    "results/WBTS_CFIN_2005_2017_occurrence_frompy.csv",
    header=True,
    index=False,
    quoting=csv.QUOTE_ALL,
    columns=columns)

 All done with occurrence!

# Measurement Or Fact
The last file we need to create is the measurement or fact file. For this we need to
combine all of the measurements or facts that we want to include making sure to include
IDs from the BODC NERC vocabulary where possible.

Now we'll manually map the remaining variables to the BODC NERC vocabulary when possible. For now we're mapping the following metadata for each field:

1. uri -> URL of the concept page on the NERC VOcabulary Server (NVS)
2. unit
3. unitId -> URL of the unit ID page on NVS
4. accuracy 
5. type -> measurement type 

In [ ]:
df.columns

vocab_url_prefix = 'http://vocab.nerc.ac.uk/collection/'

column_mappings = {
    'Cast': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Net_Type': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Mesh_Size': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'NET_DEPTH': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'STATION_DEPTH': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Plankton_Net_Area': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Volume_Filtered': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Sample_Split': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Sample_Dry_Weight': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'DW_G_M_2': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'Dilution_Factor': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
    'TOTAL_DILFACTOR_CFIN': {'uri': 'P25/current/VOL/', 'unit': 'm2', 'unitID': 'P06/current/UPAA/', 'accuracy': '', 'type': ''},
}

No we'll loop through the mapping list and transform as needed.

In [ ]:
frames_to_concat = []

for current_field in column_mappings:

    current_mapping = column_mappings.get(current_field)
    
    current_df = df[['eventID', current_field, 'time']].copy()

    current_df['occurrenceID'] = ''
    current_df['measurementType'] = current_mapping.get('type')
    current_df['measurementTypeID'] = vocab_url_prefix + current_mapping.get('uri')
    current_df['measurementValue'] = current_df[current_field]
    current_df['measurementUnit'] = current_mapping.get('unit')
    current_df['measurementUnitID'] = vocab_url_prefix + current_mapping.get('unitID')
    current_df['measurementAccuracy'] = current_mapping.get('accuracy')
    current_df['measurementDeterminedDate'] = current_df['time']
    current_df['measurementMethod'] = ''
    current_df.drop(
        columns=[current_field, 'time'],
        inplace=True)
    
    frames_to_concat.append(current_df)


Concatenate all measurements or facts together.

In [ ]:
measurementorfact = pd.concat(frames_to_concat)

Let's check to see what it looks like

In [ ]:
measurementorfact.head(50)

Write measurement or fact file

In [ ]:
measurementorfact.to_csv('results/WBTS_CFIN_2005_2017_mof_frompy.csv',
                         index=False,
                         header=True)